In [1]:
"""Getting all relevent imports into the file"""

import pandas as pd #imported pandas to use dataframes
import numpy as np #imported numpy for dynamic arrays
import matplotlib.pyplot as plt # imported pyplot to use with my visuals
import requests #imported to use webscrape
from requests import get #pulled get to use for scraping
from random import randint #pulled randint to use as a buffer for time
from time import sleep #pulled sleep to slow the crawl rate of the scraper
from bs4 import BeautifulSoup #pulled to webscrape


In [2]:
"""Pulling information from the actual IMDb website."""
pages = np.arange(1,1001,250)
title = []
directors = []
movie_rating = []
year =  []
star_rating = []
gross = []
movie_genre = []
movie_time = []
votes = []
def scraper():
    for page in pages:
        page = requests.get('https://www.imdb.com/search/title/?release_date=1700-01-01,2020-12-31&groups=top_1000&certificates=US%3AG,US%3APG,US%3APG-13,US%3AR,US%3ANC-17&sort=alpha,asc&count=250&start='+str(page)+'&ref_=adv_nxt')
        
        soup = BeautifulSoup(page.text, 'html.parser')
        
#pulling from lister-item mode-advanced pulls all the information for each of the movies on the page to easily pull 
#the Html tags as attributes of an item
        container = soup.findAll('div',class_ = 'lister-item mode-advanced') 
        """ for loop looking for each of the catagories in each of the containers"""
        for name in container:
            """nv is so we can seperate the gross income and seperate it from the votes set into their own """
            nv = name.find_all('span',attrs ={'name':'nv'})
            
            title.append(name.h3.a.text)
            movie_rating.append(name.p.span.text)
            director = name.find('p',class_='').find_all('a')[0].text
            directors.append(director)
            year.append(name.h3.find('span',class_ = 'lister-item-year').text) 
            star_rating.append(float(name.strong.text))
            gross_income = nv[1].text.strip('$\n\tM') if len(nv) > 1 else '0'
            gross.append(gross_income)
            votes_total = nv[0].text if len(nv) >1 else '0'
            votes.append(votes_total)
            movie_genres = name.p.find('span',class_ ='genre').text.strip(',\n\t" "').split(',')
            #movie_genre.append(list(map(lambda ele: ele.split(','),movie_genres)))#movie_genres
            movie_genre.append(movie_genres)
            movie_time.append(name.p.find('span', class_ = 'runtime').text)


In [3]:
"""Cleaning of the information. removing unneeded characters out of the set of information as well as setting up the dataframe"""
def Clean():
    df.loc[:,'Year'] = df['Year'].astype(str).str[-5:-1].astype(int)
    df.loc[:,'Movie Runtime'] = df['Movie Runtime'].astype(str).str[:-3].astype(int)
    df.loc[:,'Votes'] = df['Votes'].str.replace(',','').astype(float)
    df.loc[:,'Gross Income']= df['Gross Income']
    df.loc[:, 'Title'] = df['Title'].astype(str)
    df.loc[:,'Movie Genre'] = df['Movie Genre'].astype(str)
    df.loc[:, 'Director'] = df['Director'].astype(str)
    df.drop_duplicates()
    

In [4]:
"""Pull and clean information to get ready for setting to dataframe"""
scraper()


In [5]:
"""Setting the information from above to a dataframe to manipulate and visualize"""
df = pd.DataFrame({'Title': title,'Movie Rating': movie_rating,'Year': year,'Director':directors,
                   'Gross Income': gross,'Movie Genre': movie_genre, 'Movie Runtime': movie_time,
                    'IMDb Rating': star_rating,'Votes': votes})
Clean()
df.head()

,Title,Movie Rating,Year,Director,Gross Income,Movie Genre,Movie Runtime,IMDb Rating,Votes
0,500 Days of Summer,PG-13,2009,Marc Webb,32.39,"['Comedy', ' Drama', ' Romance']",95,7.7,462111.0
1,12 Years a Slave,R,2013,Steve McQueen,56.67,"['Biography', ' Drama', ' History']",134,8.1,620348.0
2,127 Hours,R,2010,Danny Boyle,18.34,"['Biography', ' Drama']",94,7.6,340655.0
3,1917,R,2019,Sam Mendes,159.23,"['Drama', ' War']",119,8.3,342958.0
4,2001: A Space Odyssey,G,1968,Stanley Kubrick,56.95,"['Adventure', ' Sci-Fi']",149,8.3,583340.0


In [6]:
df.set_index('Title')


,Movie Rating,Year,Director,Gross Income,Movie Genre,Movie Runtime,IMDb Rating,Votes
Title,,,,,,,,
500 Days of Summer,PG-13,2009,Marc Webb,32.39,"['Comedy', ' Drama', ' Romance']",95,7.7,462111.0
12 Years a Slave,R,2013,Steve McQueen,56.67,"['Biography', ' Drama', ' History']",134,8.1,620348.0
127 Hours,R,2010,Danny Boyle,18.34,"['Biography', ' Drama']",94,7.6,340655.0
1917,R,2019,Sam Mendes,159.23,"['Drama', ' War']",119,8.3,342958.0
2001: A Space Odyssey,G,1968,Stanley Kubrick,56.95,"['Adventure', ' Sci-Fi']",149,8.3,583340.0
...,...,...,...,...,...,...,...,...
Cold War,R,2018,Pawel Pawlikowski,4.57,"['Drama', ' History', ' Music']",89,7.6,44956.0
Zodiac,R,2007,David Fincher,33.08,"['Crime', ' Drama', ' Mystery']",157,7.7,438036.0
Zombieland,R,2009,Ruben Fleischer,75.59,"['Adventure', ' Comedy', ' Fantasy']",88,7.6,507818.0


In [7]:
df.to_csv("df_scraped.csv",index=False)

